In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import platform
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [156]:
# 년도별 CSV 파일들을 한꺼번에 concat하는 함수
# The function for concatenating annual CSV files.

def concat_df(year):
    ret_df = pd.DataFrame([])
    for i in range(1, 12):
        if i < 10:
            df = pd.read_csv(f'./{year}0{str(i)}_excercise.csv')
        else:
            df = pd.read_csv(f'./{year}{str(i)}_excercise.csv')
        ret_df = pd.concat([ret_df, df])
    return ret_df

In [157]:
df1 = concat_df('2021')
df2 = concat_df('2022')

In [158]:
# 2023년은 4월까지의 데이터만 존재하기 때문에 개별적으로 read 후 concat
# Data for the year 2023 only has data until April so it was necessary to read each files individually and then concatenate them

df3 = pd.read_csv('./202301_excercise.csv')
df4 = pd.read_csv('./202302_excercise.csv')
df5 = pd.read_csv('./202303_excercise.csv')
df6 = pd.read_csv('./202304_excercise.csv')

final_final_df = pd.concat([df1, df2, df3, df4, df5, df6], axis=0, ignore_index=True)

In [159]:
# final_final_df.to_csv('final_final_df.csv')

In [160]:
final_final_df.shape

(472375, 51)

In [161]:
df=final_final_df[['MESURE_AGE_CO', 'SEXDSTN_FLAG_CD','MESURE_IEM_001_VALUE','MESURE_IEM_002_VALUE','MESURE_IEM_003_VALUE',
               'MESURE_IEM_005_VALUE', 'MESURE_IEM_006_VALUE','MESURE_IEM_012_VALUE',
               'MESURE_IEM_018_VALUE','MESURE_IEM_022_VALUE','MVM_PRSCRPTN_CN','MESURE_IEM_009_VALUE','MESURE_IEM_020_VALUE',
               'MESURE_IEM_007_VALUE','MESURE_IEM_008_VALUE']]
df.columns=['나이','성별','신장','체중','체지방율','최저혈압','최고혈압',
            '좌전굴','BMI','제자리멀리뛰기','운동처방','윗몸일으키기','왕복오래달리기','악력(좌)','악력(우)']

In [162]:
df['악력'] = [n for n in range(0,df.shape[0])]
df['악력'] = df[['악력(좌)', '악력(우)']].max(axis=1)

C:\Users\KDP-23\AppData\Local\Temp\ipykernel_6100\3467385715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['악력'] = [n for n in range(0,df.shape[0])]
C:\Users\KDP-23\AppData\Local\Temp\ipykernel_6100\3467385715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['악력'] = df[['악력(좌)', '악력(우)']].max(axis=1)


In [163]:
def replace_na(df, column):
        for age in df['나이'].unique():
            age_avg = df[df['나이'] == age][column].mean()
            df.loc[df['나이'] == age, column] = df.loc[df['나이'] == age, column].fillna(age_avg)
            df.loc[df['나이'] == age, column] = df.loc[df['나이'] == age, column].replace(0, age_avg)
        return df

In [164]:
def exercise_preprocessing(df):           
    mask=df.나이 <= 64
    df=df[mask]
    df= df.drop(columns=['악력(좌)', '악력(우)', '운동처방', '윗몸일으키기'])
    # df['성별'] = df['성별'].replace('M', 0).replace('F', 1)
    df = replace_na(df, '제자리멀리뛰기')
    df = replace_na(df, '왕복오래달리기')
    df = replace_na(df, '체지방율')
    df = replace_na(df, '최저혈압')
    df = replace_na(df, '최고혈압')
    df = df.dropna(subset=['체지방율', '신장', '체중', '좌전굴', '악력', 'BMI'])
    
    return df

In [165]:
final_df = exercise_preprocessing(df)

In [166]:
final_df.isna().sum()

나이         0
성별         0
신장         0
체중         0
체지방율       0
최저혈압       0
최고혈압       0
좌전굴        0
BMI        0
제자리멀리뛰기    0
왕복오래달리기    0
악력         0
dtype: int64

In [167]:
final_df.shape[0]

410743

In [168]:
women_df = final_df[final_df['성별']=='F']

In [169]:
women_df.shape[0]

158673

In [171]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000
mean,24.593579,159.665545,56.998954,29.635811,73.817329,120.647918,14.065373,22.290683,151.988231,30.350658,25.291709
std,15.063211,6.571781,11.470048,12.932433,25.440258,34.071256,10.215919,4.002278,28.977240,13.197394,5.984706
min,11.000000,51.900000,10.000000,0.900000,3.000000,-84.000000,-47.000000,3.800000,0.247000,0.870000,0.000000
25%,15.000000,155.900000,49.700000,25.100000,67.000000,112.000000,8.600000,19.600000,132.000000,20.000000,21.800000
50%,18.000000,160.000000,55.700000,30.700000,73.441416,120.000000,15.100000,21.700000,154.000000,31.000000,25.300000
75%,28.000000,164.000000,62.800000,35.500000,80.000000,127.000000,20.600000,24.300000,174.232697,41.032553,28.800000
max,64.000000,198.800000,158.600000,3637.000000,7275.000000,11342.000000,1706.000000,171.900000,1636.000000,150.000000,834.000000


In [172]:
# 입력 잘못된 데이터들을 도메인 지식에 따라 이상치 처리하는 함수
# The function for which preprocessing outliers by domain for incorrectly inputted data

def float_num_100(value):
    if value >= 100:
        value = float((value/10000)) * 100
    return value

def float_num_1000(value):
    if value >= 1000:
        value = float((value/10000)) * 100
    return value

In [173]:
women_df['체지방율'] = women_df['체지방율'].apply(float_num_100)

C:\Users\KDP-23\AppData\Local\Temp\ipykernel_6100\2921763118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  women_df['체지방율'] = women_df['체지방율'].apply(float_num_100)


In [174]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000,158673.000000
mean,24.593579,159.665545,56.998954,29.594877,73.817329,120.647918,14.065373,22.290683,151.988231,30.350658,25.291709
std,15.063211,6.571781,11.470048,8.987140,25.440258,34.071256,10.215919,4.002278,28.977240,13.197394,5.984706
min,11.000000,51.900000,10.000000,0.900000,3.000000,-84.000000,-47.000000,3.800000,0.247000,0.870000,0.000000
25%,15.000000,155.900000,49.700000,25.100000,67.000000,112.000000,8.600000,19.600000,132.000000,20.000000,21.800000
50%,18.000000,160.000000,55.700000,30.700000,73.441416,120.000000,15.100000,21.700000,154.000000,31.000000,25.300000
75%,28.000000,164.000000,62.800000,35.500000,80.000000,127.000000,20.600000,24.300000,174.232697,41.032553,28.800000
max,64.000000,198.800000,158.600000,76.000000,7275.000000,11342.000000,1706.000000,171.900000,1636.000000,150.000000,834.000000


In [175]:
women_df = women_df[(women_df['체지방율'] <= 50) & (women_df['체지방율'] >= 8)]

In [176]:
women_df['체지방율'].shape

(157815,)

In [177]:
women_df['최저혈압'] = women_df['최저혈압'].apply(float_num_100)
women_df['최고혈압'] = women_df['최고혈압'].apply(float_num_100)

In [178]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000,157815.000000
mean,24.610062,159.666374,56.814933,29.490580,73.518834,3.737723,14.096456,22.218600,152.108740,30.395718,25.287186
std,15.075518,6.571900,11.131132,8.861343,10.019896,15.163903,10.207297,3.860531,28.882629,13.180618,5.961355
min,11.000000,51.900000,10.000000,8.100000,1.000000,-84.000000,-47.000000,3.800000,0.247000,0.870000,0.000000
25%,15.000000,155.900000,49.700000,25.000000,67.000000,1.130000,8.600000,19.600000,132.000000,20.000000,21.800000
50%,18.000000,160.000000,55.600000,30.600000,73.000000,1.200000,15.100000,21.700000,154.000000,31.000000,25.300000
75%,28.000000,164.000000,62.700000,35.400000,80.000000,1.280000,20.600000,24.300000,175.000000,41.032553,28.800000
max,64.000000,198.800000,142.900000,50.000000,99.000000,113.420000,1706.000000,171.900000,1636.000000,150.000000,834.000000


In [179]:
women_df = women_df[women_df['최저혈압'] >= 50]

In [180]:
women_df['최고혈압'] = women_df['최고혈압'].apply(lambda x : (x*100) if x< 3 else x)

In [181]:
women_df = women_df[(women_df['최고혈압'] <= 180) & (women_df['최고혈압'] > 80 )]

In [182]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,156106.00000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000
mean,24.60498,159.672645,56.831601,29.500751,73.872548,120.504451,14.102813,22.223122,152.073214,30.377964,25.291406
std,15.07642,6.570271,11.140044,8.866134,9.324639,12.517724,10.217678,3.862026,28.885099,13.184251,5.966042
min,11.00000,51.900000,10.000000,8.100000,50.000000,81.000000,-47.000000,3.800000,0.247000,0.870000,0.000000
25%,15.00000,155.900000,49.700000,25.100000,68.000000,112.000000,8.700000,19.600000,132.000000,20.000000,21.800000
50%,18.00000,160.000000,55.600000,30.600000,74.000000,120.000000,15.100000,21.700000,154.000000,31.000000,25.300000
75%,28.00000,164.000000,62.700000,35.400000,80.000000,127.000000,20.600000,24.300000,174.232697,41.032553,28.800000
max,64.00000,198.800000,142.900000,50.000000,99.000000,180.000000,1706.000000,171.900000,1636.000000,150.000000,834.000000


In [183]:
# 잘못 입력된 데이터 (컬럼: 좌전굴) 이상치 처리하는 함수
# Function to handle outliers for incorrectly entered data (Column: 좌전굴)

def sitdown_cave(value):
    if value > 60:
        value = float((value/10000)) * 100
    if value < -40:
        value = float(value/10)
    return value

In [184]:
women_df['좌전굴'] = women_df['좌전굴'].apply(sitdown_cave)

In [185]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,156106.00000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000
mean,24.60498,159.672645,56.831601,29.500751,73.872548,120.504451,14.081168,22.223122,152.073214,30.377964,25.291406
std,15.07642,6.570271,11.140044,8.866134,9.324639,12.517724,9.172754,3.862026,28.885099,13.184251,5.966042
min,11.00000,51.900000,10.000000,8.100000,50.000000,81.000000,-30.000000,3.800000,0.247000,0.870000,0.000000
25%,15.00000,155.900000,49.700000,25.100000,68.000000,112.000000,8.700000,19.600000,132.000000,20.000000,21.800000
50%,18.00000,160.000000,55.600000,30.600000,74.000000,120.000000,15.100000,21.700000,154.000000,31.000000,25.300000
75%,28.00000,164.000000,62.700000,35.400000,80.000000,127.000000,20.600000,24.300000,174.232697,41.032553,28.800000
max,64.00000,198.800000,142.900000,50.000000,99.000000,180.000000,57.000000,171.900000,1636.000000,150.000000,834.000000


In [186]:
# 잘못 입력된 데이터 (컬럼: 제자리멀리뛰기) 이상치 처리하는 함수
# Function to handle outliers for incorrectly entered data (Column: 제자리멀리뛰기)
def Board_jump(value):
    if value >= 300:
        value = float((value/10000)) * 100
    return value

In [187]:
# 잘못 입력된 데이터 (컬럼: BMI) 이상치 처리하는 함수
# Function to handle outliers for incorrectly entered data (Column: BMI)
def BMI_score(value):
    if value >= 50:
        value = float((value/10000)) * 100
    return value

# 잘못 입력된 데이터 (컬럼: 악력) 이상치 처리하는 함수
# Function to handle outliers for incorrectly entered data (Column: 악력)
def Grab_strength(value):
    if value > 80:
        value = float((value/10000)) * 100
    return value

In [188]:
women_df['좌전굴'] = women_df['좌전굴'].apply(sitdown_cave)
women_df['BMI'] = women_df['BMI'].apply(BMI_score)
women_df['제자리멀리뛰기'] = women_df['제자리멀리뛰기'].apply(Board_jump)
women_df['악력'] = women_df['악력'].apply(Grab_strength)

In [189]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,156106.00000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000,156106.000000
mean,24.60498,159.672645,56.831601,29.500751,73.872548,120.504451,14.081168,22.218265,152.038081,30.377964,25.276953
std,15.07642,6.570271,11.140044,8.866134,9.324639,12.517724,9.172754,3.801620,28.194226,13.184251,5.456380
min,11.00000,51.900000,10.000000,8.100000,50.000000,81.000000,-30.000000,0.533000,0.247000,0.870000,0.000000
25%,15.00000,155.900000,49.700000,25.100000,68.000000,112.000000,8.700000,19.600000,132.000000,20.000000,21.800000
50%,18.00000,160.000000,55.600000,30.600000,74.000000,120.000000,15.100000,21.700000,154.000000,31.000000,25.300000
75%,28.00000,164.000000,62.700000,35.400000,80.000000,127.000000,20.600000,24.300000,174.232697,41.032553,28.800000
max,64.00000,198.800000,142.900000,50.000000,99.000000,180.000000,57.000000,46.600000,281.000000,150.000000,79.900000


In [190]:
women_df = women_df[women_df['왕복오래달리기'] < 80] 
women_df = women_df[women_df['제자리멀리뛰기'] >= 50]
women_df = women_df[women_df['BMI'] >= 15]


In [191]:
women_df.shape[0]

153927

In [192]:
women_df = women_df[women_df['악력'] > 5] 

In [193]:
women_df.shape[0]

153914

In [194]:
women_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000,153914.000000
mean,24.768020,159.809067,57.099094,29.732022,73.890691,120.562353,14.129872,22.298447,152.148996,29.903563,25.377289
std,15.099671,6.396451,10.915464,8.662698,9.347352,12.547983,9.170748,3.744892,28.045835,12.146884,5.379842
min,11.000000,117.900000,21.300000,8.200000,50.000000,81.000000,-30.000000,15.000000,50.000000,0.870000,5.100000
25%,15.000000,156.000000,49.900000,25.300000,67.000000,112.000000,8.700000,19.700000,132.000000,20.000000,21.900000
50%,18.000000,160.100000,55.800000,30.800000,74.000000,120.000000,15.200000,21.700000,154.000000,30.416149,25.400000
75%,28.000000,164.000000,62.900000,35.400000,80.000000,127.000000,20.700000,24.300000,175.000000,41.000000,28.800000
max,64.000000,198.800000,142.900000,50.000000,99.000000,180.000000,57.000000,46.600000,281.000000,79.000000,79.900000


In [195]:
men_df = pd.read_csv('./man(notyet)df.csv')

In [196]:
men_df.columns

Index(['Unnamed: 0', '나이', '성별', '신장', '체중', '체지방율', '최저혈압', '최고혈압', '좌전굴',
       'BMI', '제자리멀리뛰기', '왕복오래달리기', '악력'],
      dtype='object')

In [197]:
men_df.head()

,Unnamed: 0,나이,성별,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
0,0,29.0,0,166.8,80.9,25.2,75.0,152.0,20.9,29.1,2.012421,44.304110,49.8
1,1,58.0,1,149.9,56.6,39.1,82.0,149.0,16.8,25.2,1.200000,17.954198,22.5
2,3,21.0,0,176.6,61.1,16.0,79.0,117.0,7.8,19.6,2.000000,43.036984,34.7
3,4,23.0,0,169.5,66.7,22.4,76.0,123.0,5.4,23.2,2.200000,41.032553,41.0
4,5,21.0,1,159.6,62.6,41.6,74.0,117.0,21.1,24.6,1.310000,43.036984,21.2


In [198]:
men_df = men_df.drop(columns='Unnamed: 0')

In [199]:
men_df['성별'] = men_df['성별'].replace(0,'M').replace(1,'F')

In [200]:
men_df['제자리멀리뛰기'] = men_df['제자리멀리뛰기'].apply(lambda x : x*100 if x <3 else x)
men_df = men_df[men_df['제자리멀리뛰기'] >= 50]
men_df = men_df[(men_df['체지방율'] <= 45) & (men_df['체지방율'] >= 5)]

In [201]:
men_df['제자리멀리뛰기'] 

0         201.242129
1         120.000000
2         200.000000
3         220.000000
4         131.000000
             ...    
410738     98.000000
410739    132.000000
410740    196.000000
410741     95.000000
410742    133.000000
Name: 제자리멀리뛰기, Length: 405345, dtype: float64

In [202]:
women_df.columns

Index(['나이', '성별', '신장', '체중', '체지방율', '최저혈압', '최고혈압', '좌전굴', 'BMI', '제자리멀리뛰기',
       '왕복오래달리기', '악력'],
      dtype='object')

In [203]:
women_df.reset_index(drop=True, inplace=True)

In [204]:
men_df = men_df[(men_df['최고혈압'] <= 180) & (men_df['최고혈압'] > 80 )]
men_df = men_df[men_df['최저혈압'] >= 50]

In [205]:
men_df.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000,404910.000000
mean,24.292304,166.925453,65.518197,24.149580,75.715608,124.469645,10.762032,23.299047,178.008838,35.703984,34.803405
std,14.082345,9.713691,15.159645,9.041465,9.821007,13.582976,9.655197,4.000944,36.044364,15.474390,11.385352
min,11.000000,112.900000,11.200000,5.000000,50.000000,81.000000,-33.000000,4.200000,50.000000,0.326000,0.000000
25%,15.000000,160.300000,54.500000,17.100000,69.000000,116.000000,4.900000,20.400000,155.000000,24.000000,25.500000
50%,18.000000,167.800000,64.400000,24.300000,76.000000,122.000000,11.600000,23.000000,178.357520,36.000000,34.200000
75%,28.000000,174.100000,75.200000,31.100000,82.000000,133.000000,17.600000,25.700000,202.000000,42.428074,43.600000
max,64.000000,202.500000,174.400000,45.000000,119.000000,180.000000,57.000000,49.000000,299.000000,172.000000,87.900000


In [206]:
final = pd.concat([men_df, women_df], axis=0, ignore_index=True)

In [207]:
final.나이.unique()

array([29., 58., 21., 23., 62., 18., 19., 53., 24., 25., 28., 31., 20.,
       63., 64., 46., 61., 55., 36., 56., 26., 27., 22., 50., 30., 59.,
       51., 52., 54., 57., 33., 34., 48., 60., 35., 37., 32., 44., 42.,
       49., 47., 39., 11., 13., 40., 43., 17., 15., 14., 38., 45., 41.,
       16.])

In [208]:
final.to_csv('final_final_final.csv')

In [209]:
final.shape

(558824, 12)

In [210]:
final.corr()

C:\Users\KDP-23\AppData\Local\Temp\ipykernel_6100\3701051463.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  final.corr()


,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
나이,1.000000,0.019396,0.140613,0.197161,0.206977,0.338044,0.076871,0.193433,-0.093457,-0.327376,0.143645
신장,0.019396,1.000000,0.690487,-0.119821,0.134924,0.231451,-0.122073,0.273241,0.579514,0.146972,0.757270
체중,0.140613,0.690487,1.000000,0.278837,0.243503,0.360908,-0.110432,0.877228,0.339871,-0.029574,0.683670
체지방율,0.197161,-0.119821,0.278837,1.000000,0.080530,0.073153,0.080003,0.474803,-0.341106,-0.450411,-0.195360
최저혈압,0.206977,0.134924,0.243503,0.080530,1.000000,0.618624,-0.013117,0.233952,0.069347,-0.060421,0.199062
최고혈압,0.338044,0.231451,0.360908,0.073153,0.618624,1.000000,-0.008924,0.329311,0.137559,-0.081749,0.333891
좌전굴,0.076871,-0.122073,-0.110432,0.080003,-0.013117,-0.008924,1.000000,-0.055649,0.040504,0.007942,-0.027588
BMI,0.193433,0.273241,0.877228,0.474803,0.233952,0.329311,-0.055649,1.000000,0.071705,-0.153756,0.412947
제자리멀리뛰기,-0.093457,0.579514,0.339871,-0.341106,0.069347,0.137559,0.040504,0.071705,1.000000,0.385530,0.657633
왕복오래달리기,-0.327376,0.146972,-0.029574,-0.450411,-0.060421,-0.081749,0.007942,-0.153756,0.385530,1.000000,0.196103


In [211]:
final.describe()

,나이,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
count,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000,558824.000000
mean,24.423328,164.965423,63.199367,25.687123,75.212981,123.393480,11.689619,23.023457,170.886396,34.106405,32.207219
std,14.371288,9.473293,14.610938,9.280099,9.727076,13.419938,9.642318,3.957407,35.936896,14.861243,10.937421
min,11.000000,112.900000,11.200000,5.000000,50.000000,81.000000,-33.000000,4.200000,50.000000,0.326000,0.000000
25%,15.000000,158.500000,52.700000,19.000000,69.000000,115.000000,5.800000,20.200000,147.000000,22.000000,24.000000
50%,18.000000,164.700000,61.300000,26.400000,75.000000,121.000000,12.600000,22.600000,173.226823,34.659400,29.700000
75%,28.000000,172.000000,72.300000,32.700000,81.000000,131.000000,18.600000,25.400000,195.000000,42.428074,40.300000
max,64.000000,202.500000,174.400000,50.000000,119.000000,180.000000,57.000000,49.000000,299.000000,172.000000,87.900000


In [212]:
final.head(20)

,나이,성별,신장,체중,체지방율,최저혈압,최고혈압,좌전굴,BMI,제자리멀리뛰기,왕복오래달리기,악력
0,29.0,M,166.8,80.9,25.2,75.0,152.0,20.9,29.1,201.242129,44.304110,49.8
1,58.0,F,149.9,56.6,39.1,82.0,149.0,16.8,25.2,120.000000,17.954198,22.5
2,21.0,M,176.6,61.1,16.0,79.0,117.0,7.8,19.6,200.000000,43.036984,34.7
3,23.0,M,169.5,66.7,22.4,76.0,123.0,5.4,23.2,220.000000,41.032553,41.0
4,21.0,F,159.6,62.6,41.6,74.0,117.0,21.1,24.6,131.000000,43.036984,21.2
5,23.0,M,167.1,68.0,28.1,79.0,132.0,12.1,24.4,194.798914,41.032553,39.2
6,62.0,M,163.4,80.3,25.0,96.0,142.0,17.9,30.1,165.000000,16.924538,52.8
7,18.0,M,185.1,85.9,15.8,71.0,130.0,7.9,25.1,180.410203,36.453559,44.1
8,19.0,M,179.0,75.8,21.3,71.0,118.0,10.5,23.7,175.000000,42.428074,40.6
9,29.0,M,175.0,78.8,24.6,73.0,119.0,-2.1,25.7,198.000000,44.304110,45.9


In [213]:
from sklearn.model_selection import train_test_split

In [214]:
train_data, test_data = train_test_split(final, test_size=0.2,random_state=10)

In [215]:
# AutoMl로 머신러닝학습 코드
# The code for Machine Learning using AutoMl

from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='나이',problem_type='regression',eval_metric='r2')
predictor.fit(train_data=train_data, presets= 'medium_quality') 

In [216]:
results = predictor.fit_summary()
print(results) 

In [217]:
ld_board = predictor.leaderboard(train_data, silent=True)
ld_board

In [218]:
predictor.get_model_best()

In [219]:
predictor.evaluate(train_data)

In [220]:
from sklearn.metrics import r2_score,mean_squared_error

test_y_ = test_data['나이']
pred_y_ = predictor.predict(test_data)

r2_score(test_y_,pred_y_),mean_squared_error(test_y_,pred_y_)

In [221]:
# 모델 파일 불러오기
# read model files

saved_model_path = r'AutogluonModels/ag-20230901_103730'
predictor = TabularPredictor.load(saved_model_path)

In [222]:
# ld_board = predictor.leaderboard(train_data, silent=True)

# ld_board
# ------------------------------------------------------------------

In [223]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [224]:
# predictor.evaluate(train_data)

In [225]:
# predictor.evaluate(test_data)

In [226]:
# pred_df_test=predictor.predict(df_test)
# round(pred_df_test)

In [227]:
# 새로운 데이터 입력
# Enter new data

try:
    height = int(input("키를 입력하세요 (단위:cm) : "))
    gender = input('성별을 입력하세요 : (M or F): ')
    weight = int(input("몸무게를 입력하세요 (단위:kg) : "))
    fat_ratio = int(input("체지방율을 입력하세요 (단위:%) : "))
    low_bp = int(input("최저혈압을 입력하세요 : "))
    high_bp = int(input("최고혈압을 입력하세요 : "))
    stretch = int(input("좌전굴 최고 점수를 입력하세요 : "))
    bmi_score = int(input("BMI 점수를 입력하세요 : "))
    long_jump = int(input("제자리멀리뛰기 최고기록을 입력하세요 (단위:cm): "))
    circular_running = int(input("왕복오래달리기 최고기록을 입력하세요 (단위:회): "))
    grip_strength = int(input("좌우 악력 중 악력 최고기록을 입력하세요 (단위:kg): "))
except Exception:
    print('다시 입력하세요: ')

In [228]:
final.columns

Index(['나이', '성별', '신장', '체중', '체지방율', '최저혈압', '최고혈압', '좌전굴', 'BMI', '제자리멀리뛰기',
       '왕복오래달리기', '악력'],
      dtype='object')

In [229]:
new_data = pd.DataFrame({'성별':[gender], 
                         '신장':[height],
                         '체중':[weight], 
                         '체지방율':[fat_ratio], 
                         '최저혈압':[low_bp], 
                         '최고혈압':[high_bp],
                         '좌전굴': [stretch], 
                         'BMI':[bmi_score], 
                         '제자리멀리뛰기':[long_jump], 
                         '왕복오래달리기':[circular_running], 
                         '악력':[grip_strength]}, index=[0])

In [230]:
pred_y_WeightedEnsemble_L2_FULL = predictor.predict(new_data)

In [231]:
# "운동나이" 예측하는 모델
# ML model for predicting "Fitness Age"

f"당신의 운동 나이는 : {round(pred_y_WeightedEnsemble_L2_FULL[0])}세 입니다."

'당신의 운동 나이는 : 24세 입니다.'